PASSO 1 — Criar o schema GOLD

In [0]:
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;

PASSO 2 — Dimensão TEMPO

In [0]:
CREATE OR REPLACE TABLE gold.dim_tempo AS
SELECT DISTINCT
  ano,
  mes,
  AnoMes AS ano_mes
FROM silver.transacoes_pix;

num_affected_rows,num_inserted_rows


PASSO 3 — Dimensão REGIÃO

In [0]:
CREATE OR REPLACE TABLE gold.dim_regiao AS
SELECT DISTINCT
  pag_regiao AS regiao
FROM silver.transacoes_pix
WHERE pag_regiao IS NOT NULL

UNION

SELECT DISTINCT
  rec_regiao AS regiao
FROM silver.transacoes_pix
WHERE rec_regiao IS NOT NULL;

num_affected_rows,num_inserted_rows


PASSO 4 — Dimensão TIPO DE PESSOA

In [0]:
CREATE OR REPLACE TABLE gold.dim_tipo_pessoa AS
SELECT DISTINCT pag_tipo_pessoa AS tipo_pessoa
FROM silver.transacoes_pix

UNION

SELECT DISTINCT rec_tipo_pessoa AS tipo_pessoa
FROM silver.transacoes_pix;


num_affected_rows,num_inserted_rows


PASSO 5 — Dimensão FAIXA ETÁRIA

In [0]:
CREATE OR REPLACE TABLE gold.dim_faixa_etaria AS
SELECT DISTINCT pag_faixa_etaria AS faixa_etaria
FROM silver.transacoes_pix

UNION

SELECT DISTINCT rec_faixa_etaria AS faixa_etaria
FROM silver.transacoes_pix;

num_affected_rows,num_inserted_rows


PASSO 6 — Tabela FATO

In [0]:
CREATE OR REPLACE TABLE gold.fato_transacoes_pix AS
SELECT
  -- Tempo
  s.ano,
  s.mes,
  s.AnoMes AS ano_mes,

  -- Dimensões
  s.pag_regiao,
  s.rec_regiao,
  s.pag_tipo_pessoa,
  s.rec_tipo_pessoa,
  s.pag_faixa_etaria,
  s.rec_faixa_etaria,

  -- Métricas
  s.valor_reais,
  s.quantidade_total

FROM silver.transacoes_pix s;


num_affected_rows,num_inserted_rows


Ver tabelas

In [0]:
SHOW TABLES IN gold;

database,tableName,isTemporary
gold,dim_faixa_etaria,false
gold,dim_regiao,false
gold,dim_tempo,false
gold,dim_tipo_pessoa,false
gold,fato_transacoes_pix,false


Conferir faixa etária

In [0]:
SELECT * FROM gold.dim_faixa_etaria;


faixa_etaria
0-19
20-29
Nao se aplica
50-59
30-39
NAO_INFORMADO
40-49
60+


Conferir região

In [0]:
SELECT * FROM gold.dim_regiao;


regiao
Nao informado
SUL
SUDESTE
NORTE
NORDESTE
CENTRO-OESTE


Conferir tempo

In [0]:
SELECT * FROM gold.dim_tempo;


ano,mes,ano_mes
2024,10,202410
2025,8,202508
2021,10,202110
2022,6,202206
2020,11,202011
2021,11,202111
2021,4,202104
2025,1,202501
2023,8,202308
2025,4,202504


Conferir tipo pessoa

In [0]:
SELECT * FROM gold.dim_tipo_pessoa;


tipo_pessoa
PJ
Nao disponivel
PF


Conferir fato

In [0]:
SELECT *
FROM gold.fato_transacoes_pix
LIMIT 10;


ano,mes,ano_mes,pag_regiao,rec_regiao,pag_tipo_pessoa,rec_tipo_pessoa,pag_faixa_etaria,rec_faixa_etaria,valor_reais,quantidade_total
2022,11,202211,NORTE,NORDESTE,PF,PF,20-29,20-29,7626929.67,66474
2023,3,202303,NORTE,NORTE,PF,PJ,60+,Nao se aplica,7525.59,16
2023,1,202301,SUDESTE,Nao informado,PJ,PF,Nao se aplica,30-39,255424.33,1194
2024,1,202401,SUL,NORDESTE,PJ,PJ,Nao se aplica,Nao se aplica,668315.98,77
2024,8,202408,NORDESTE,SUL,PF,PF,50-59,20-29,1226112.51,8636
2023,7,202307,NORTE,Nao informado,PF,PF,50-59,40-49,23848.56,136
2022,10,202210,SUL,SUDESTE,PF,PF,50-59,30-39,58934.01,910
2022,11,202211,CENTRO-OESTE,NORTE,PF,PF,50-59,30-39,24371515.36,64311
2022,11,202211,CENTRO-OESTE,SUDESTE,PJ,PJ,Nao se aplica,Nao se aplica,70869.33,14
2022,10,202210,NORTE,NORTE,PF,PJ,60+,Nao se aplica,37606858.96,53690


Agora vamos responder as perguntas de negócio

## PF → PF ≥ 100 bilhões

In [0]:
SELECT
  ano_mes,
  ROUND(SUM(valor_reais) / 1000000000, 2) AS valor_em_bilhoes
FROM gold.fato_transacoes_pix
WHERE
  pag_tipo_pessoa = 'PF'
  AND rec_tipo_pessoa = 'PF'
GROUP BY ano_mes
HAVING SUM(valor_reais) >= 1000000000
ORDER BY ano_mes;


ano_mes,valor_em_bilhoes
202011,12.05
202012,48.81
202101,61.98
202102,76.01
202103,101.87
202104,116.03
202105,138.06
202106,152.61
202107,177.15
202108,189.65


## PF → PJ ≥ 100 bilhões

In [0]:
SELECT
  ano_mes,
  ROUND(SUM(valor_reais) / 1000000000, 2) AS valor_em_bilhoes
FROM gold.fato_transacoes_pix
WHERE
  pag_tipo_pessoa = 'PF'
  AND rec_tipo_pessoa = 'PJ'
GROUP BY ano_mes
HAVING SUM(valor_reais) >= 1000000000
ORDER BY ano_mes;


ano_mes,valor_em_bilhoes
202011,1.44
202012,6.29
202101,10.74
202102,14.56
202103,21.55
202104,26.85
202105,33.36
202106,36.28
202107,42.45
202108,47.63


## Distribuição de valor médio por região (pagador - em milhões)

In [0]:
SELECT
  pag_regiao,
  ROUND(AVG(valor_reais) / 1000000, 2) AS valor_medio_milhoes
FROM gold.fato_transacoes_pix
GROUP BY pag_regiao
ORDER BY valor_medio_milhoes DESC;


pag_regiao,valor_medio_milhoes
SUDESTE,373.62
SUL,129.64
NORDESTE,112.49
CENTRO-OESTE,71.96
NORTE,43.44
Nao informado,0.82


## Regiões que mais enviam - em milhões

In [0]:
SELECT
  pag_regiao,
  ROUND(SUM(valor_reais) / 1000000, 2) AS total_enviado_milhoes
FROM gold.fato_transacoes_pix
GROUP BY pag_regiao
ORDER BY total_enviado_milhoes DESC;


pag_regiao,total_enviado_milhoes
SUDESTE,40502348.71
SUL,13015577.70
NORDESTE,11917614.95
CENTRO-OESTE,7302353.25
NORTE,4375012.21
Nao informado,67215.95


## Regiões que mais receberam - em milhões

In [0]:
SELECT
  rec_regiao,
  ROUND(SUM(valor_reais) / 1000000, 2) AS total_recebido_milhoes
FROM gold.fato_transacoes_pix
GROUP BY rec_regiao
ORDER BY total_recebido_milhoes DESC;


rec_regiao,total_recebido_milhoes
SUDESTE,40364193.65
SUL,13017789.74
NORDESTE,11761359.53
CENTRO-OESTE,7645524.44
NORTE,4313125.71
Nao informado,78129.70
